In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPClassifier
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [6]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
from tqdm import tqdm

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/train.csv').fillna(' ')
test = pd.read_csv('../input/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']

def correlation(a , b ,unwanted='*'):
	a = list(a.lower())
	b = list(b.lower())
	
	count = 0 
	flag =0
	for i , j in zip(a,b) :
		if (not flag) and j not in unwanted:
			flag =1
		if i == j or j in unwanted :
			count = count + 1
			continue

		return 0
	return count >= len(a)/2 and flag
print(train.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [ ]:

def remove_unwanted_char(arr):
    badWords = ["fuck" ,"cock" ,"suck" ,"piss" ,"bullshit" ,"ass" ,"asshole" ,"dick","shit" ,"motherfuck"]
    for k in range(len(arr)) :
        tmp = arr[k].split()
        flag = 0
        for j in badWords:
            for i in range(len(tmp)):
                if correlation(j ,tmp[i]) :
                    tmp[i]= j
                    flag =1
                    
        if flag :
            arr[k] = " ".join(tmp)
    return arr
#train_text = remove_unwanted_char(train_text)
#test_text = remove_unwanted_char(test_text)

In [ ]:
print(np.shape([train_text, test_text]))

In [ ]:


all_text = pd.concat([train_text, test_text])
 

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=30000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)



losses = []

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}.',
    ngram_range=(1, 1),
    max_features=40000,
    max_df =.4 ,
    min_df    = .000001
)
#word_vectorizer.fit(all_text)
#train_word_features = word_vectorizer.transform(train_text)
#test_word_features = word_vectorizer.transform(test_text)
train_features = hstack([train_char_features])
test_features = hstack([test_char_features])


In [ ]:

predictions = {'id': test['id']}
for class_name in tqdm(class_names):
#     if class_name == "threat" :
        
    train_target = train[class_name]
    #classifier = LogisticRegression(solver='sag',multi_class="ovr",tol=1e-10)
    classifier =MLPClassifier(solver='lbfgs', alpha=1e-5,      hidden_layer_sizes=(15,), random_state=1)
    cv_loss = np.mean(cross_val_score(classifier,train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))

    classifier.fit(train_features, train_target)
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]
    
print('Total CV score is {}'.format(np.mean(losses)))
submission = pd.DataFrame.from_dict(predictions)
submission.to_csv('submission.csv', index=False)

 

In [ ]:
print((train_features.toarray))

In [ ]:
# from scipy import sparse
# sparse.save_npz('train_featureMatrix.npz', train_features)
# sparse.save_npz('test_featureMatrix.npz', test_features)

# #Load
# #data = sparse.load_npz("data_sparse.npz")
# np.savetxt('train_featureMatrix.csv',train_features,delimiter=',' , fmt='%.10f' )
# np.savetxt('test_featureMatrix.csv',test_features,delimiter=',' , fmt='%.10f' )